In [8]:
import mido
import numpy as np
import matplotlib.pyplot as plt
import Parser
import PreProcessor as pp

In [9]:
dataset = pp.load_dataset("adl-piano-midi")
Songs = pp.files_to_songs(dataset["Classical"])

channel_to_ind, ind_to_channel, note_to_ind, ind_to_note, velocity_to_ind, ind_to_velocity, tick_to_ind, ind_to_tick = pp.dicts_from_songs(Songs)

n_Channels = len(channel_to_ind)
n_Notes = len(note_to_ind)
n_Velocities = len(velocity_to_ind)
n_Ticks = len(tick_to_ind)

print("Number of channels  :",n_Channels,"\nNumber of notes     :",n_Notes,"\nNumber of velocities:",n_Velocities,"\nNumber of time ticks:",n_Ticks)

adl-piano-midi\adl-piano-midi\Classical\Classical\Alexander Borodin\Nocturne.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Antonio Salieri\Invention.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Bela Bartok\Andante.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Bela Bartok\Romanian Folk Dance.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Claude Debussy\Clair De Lune.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Franz Schubert\Impromptu in C-Moll.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Franz Schubert\Impromptu in Ges-dur.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Franz Schubert\Trois Marches militaires 3.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Frederic Chopin\Prelude Op 28 No7.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Giacomo Puccini\O Mio Babibino Caro.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Jean-Baptiste Lully\Minuet.mid
adl-piano-midi\adl-piano-midi\Classical\Classical\Johann Sebas

In [11]:
seq_length=25
n_batch=64

AllNotes = np.array([])
for song in Songs.values():
    AllNotes = np.append(AllNotes,song[:,1])
OneHotNotes = pp.one_hot_encode(note_to_ind,AllNotes)
X_train, y_train = pp.label_sequences(OneHotNotes,seq_length,n_batch)
print(X_train.shape)

(88512, 25, 105)


In [3]:
#synthetize

def synthesize_model(model, x_0, n):
    samples = []
    x = np.copy(x_0)
    for i in range(n):
        prediction = model.predict(x[np.newaxis, :, :])  # Prédire la note suivante
        samples.append(prediction)
        x = np.concatenate((x[1:, :], prediction), axis=0)  # Mettre à jour la séquence d'entrée avec la prédiction
    return np.array(samples)

In [ ]:
sample=synthesize_model(model,training_one_hot[:, 0], 100)

In [4]:
from keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN
from keras import optimizers

In [ ]:
dim=len(note_to_ind)

In [12]:
def create_weight(notes):
    # Calculer les poids en fonction de la fréquence des notes
    note_counts = {}
    for note in notes:
        for element in note:
            if element in note_counts:
                note_counts[element] += 1
            else:
                note_counts[element] = 1
    
    # Calculer les poids pour chaque note
    total_notes = sum(note_counts.values())
    weights = {note: count/total_notes for note, count in note_counts.items()}
    
    return weights

In [ ]:
def create_model(dim,weights,training_one_hot,label_one_hot):
    model = Sequential()
    model.add(SimpleRNN(units=dim, input_shape=(5,dim)))
    model.add(Dense(units=dim, activation='softmax'))  # Ajouter la couche de sortie
    model.summary()
    # Compiler le modèle
    opt = optimizers.Adagrad(learning_rate=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    # Train the model
    model.fit(training_one_hot, label_one_hot, epochs=20, batch_size=32,class_weight=weights)
    return model

In [ ]:
def generate_sequence(epoch_num, initial_index, seq_length,ind_to_note,dim,weights):

    
    model = create_model(dim)
    weights = create_weight(notes)
    model.load_weights(model_weights_directory + "weights.{}.h5".format(epoch_num))
     
    sequence_index = [initial_index]
    notes=[]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        raw_predictions = model.predict(input_data)
        predicted_probs = np.softmax(raw_predictions, axis=-1)
        sample = np.random.choice(dim, size = 1, p = predicted_probs)
        sequence_index.append(sample[0])
    
    notes = []
    
    
    return seq2

In [ ]:
ep=90
ini=65#initial note
length=2000 #size of the sequence

music = generate_sequence(ep,ini,length)
